# Reconocimiento facial

En este notebook vamos a construir un sistema de reconocimiento facial. La mayoría de las ideas presentadas aquí provienen de [FaceNet](https://arxiv.org/pdf/1503.03832.pdf). Otra red que se utiliza habitualmente para el reconocimiento facial es [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf). 

Los problemas de reconocimiento facial se pueden clasificar en dos categorías.

- La **verificación facial** intenta responder a la pregunta ¿es esta persona quien dice ser?. Por ejemplo, en algunos aeropuertos (sobre todo en China) es posible pasar algunos controles mediante un sistema que escanéa el pasaporte y verifica que el portador de dicho pasaporte es el propietario del mismo. Una aplicación móvil que se desbloquea usando tu cara también usa verificación facial. Es decir, este es un problema de emparejamiento 1 a 1. 
- El **reconocimiento facial** intenta responder a la pregunta ¿quién es esta persona? Por ejemplo, la empresa [Baidu](https://www.youtube.com/watch?v=wr4rx0Spihs) permite a sus empleados entrar a la oficina sin necesidad de identificarse. Esto es un problema de emparejamiento 1 a K. 

FaceNet es una red neuronal que aprende a codificar la imagen de una cara en un vector de 128 números. Al comparar dos de esos vectores se puede determinar si dos imágenes son de la misma persona. 

En este notebook aprenderás:
- La definición de la función de pérdida triple.
- Cómo usar un modelo entrenado para asignar imágenes de caras a una codificación en forma de un vector de tamaño 128.
- Cómo usar esas codificaciones para llevar a cabo verificación facial y reconocimiento facial. 

## 0.  Carga de paquetes
Comenzamos cargando los paquetes necesarios. Posiblemente necesites instalar alguno de ellos utilizando `pip`. El paquete más difícil de instalar es OpenCV, y las instrucciones para la instalación de dicho paquete las puedes encontrar en el aula virtual. 

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1 - Verificación facial ingenua

En el problema de la verificación facial se proporcionan dos imágenes y el programa tiene que decidir si pertenecen a la misma persona. La manera más sencilla de hacer esto es comparar las dos imágenes pixel a pixel. Si la distancia entre las dos imágenes es menor que un determinado umbral, entonces las dos imágenes pertenecen a la misma persona.

<img src="images/pixel_comparison.png" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> **Figura 1** </u></center></caption>

Como puedes imaginar, este algoritmo no funciona demasiado bien, dado que los valores de píxeles cambian de forma dramática dependiendo de las condiciones de luz, orientación de la cara de la persona, pequeños cambios en la inclinación de la cabeza, etc. 

Por lo tanto, en lugar de usar directamente la imagen, es mejor utilizar una codificación $f(img)$ de manera que la comparación de imágenes representadas mediante esa codificación permita decidir si las dos imágenes pertenecen a la misma persona.  

## 2 - Codificando las imágenes faciales usando un vector de 128 componentes

### 2.1 - Usando una red para calcular las codificaciones

El modelo FaceNet utiliza una gran cantidad de datos y lleva bastante tiempo entrenarlo. Por lo que una solución que se suele aplicar normalmente consiste en cargar los pesos de una red que ya ha sido entrenada. La red que utilizamos en este caso es el modelo Inception  [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). Esta red está disponible en el fichero `inception_blocks.py` pero no es necesario entrar en detalle de cómo funciona esta red para este notebook. El modelo Inception fue entrenado para clasificar imágenes en 1000 categorías, para ser capaces de producir una codificación de 128 componentes lo que se hace es eliminar la última capa de la red y utilizar la penúltima capa (que tiene 128 nodos) como salida. 


Los aspectos clave a tener en cuenta son:
- La red usa imágenes RGB de tamaño 96x96 como entrada. Concretamente toma una imagen de una cara (o lote de $m$ imágenes) con la forma $(m, 3, 96, 96)$ (si hay solo una imagen $m=1$).
- La red produce como salida una matriz de la forma $(m, 128)$ que codifica cada una de las imágenes que ha recibido como entrada en un vector de 128 componentes. 

Ejecuta la siguiente celda para crear el modelo de las caras.

In [25]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [26]:
print("Parametros totales de la red:", FRmodel.count_params())

('Parametros totales de la red:', 3743280)


In [27]:
print("Estructura de la red")
FRmodel.summary()

Estructura de la red
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 3, 96, 96)     0                                            
____________________________________________________________________________________________________
zero_padding2d_93 (ZeroPadding2D (None, 3, 102, 102)   0           input_5[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 48, 48)    9472        zero_padding2d_93[0][0]          
____________________________________________________________________________________________________
bn1 (BatchNormalization)         (None, 64, 48, 48)    256         conv1[0][0]                      
______________________________________________________________________

Ahora podemos usar la salida de la red para comparar dos imágenes de la siguiente manera.

<img src="images/distance_kiank.png" style="width:680px;height:250px;">
<caption><center> <u> <font color='purple'> **Figura 2**: <br> </u> <font color='purple'>Calculando la distancia entre las dos codificaciones y usando un umbral se puede determinar si las dos imágenes representan a la misma persona.</center></caption>

Así que una buena codificación es aquella en la cual:
- Si dos imágenes pertenecen a la misma persona, las codificaciones de dichas imágenes serán similares. 
- Si dos imágenes pertenecen a personas distintas, las codificaciones de dichas imágenes serán diferentes.

La función que se suele utilizar para medir esta distancia es la pérdida triple, la cual se intenta acercar las codificaciones de las dos imágenes de una misma persona (imagen ancla e imagen positiva), mientras que intenta alejar las codificaciones de dos personas distintas (imagen ancla e imagen negativa).

<img src="images/triplet_comparison.png" style="width:280px;height:150px;">
<br>
<caption><center> <u> <font color='purple'> **Figura 3**: <br> </u> <font color='purple'>De izquierda a derecha: imagen Ancla (A), imagen Positiva (P), imagen Negativa (N) </center></caption>



### 2.2 - La pérdida triple

Para una imagen $x$, denotaremos a su codificación como $f(x)$ donde $f$ es la función construida por la red neuronal. 

<img src="images/f_x.png" style="width:380px;height:150px;">

<!--
We will also add a normalization step at the end of our model so that $\mid \mid f(x) \mid \mid_2 = 1$ (means the vector of encoding should be of norm 1).
!-->

Para el entrenamiento usaremos tríos de imágenes $(A, P, N)$:  

- A es la imagen "Ancla": una imagen de una persona. 
- P es una imagen "Positiva": una imagen de la misma persona representada en la imagen ancla.
- N es una imagen "Negativa": una imagen de una persona distinta a la de la imagen ancla.

Estos tríos se van tomando de nuestro conjunto de entrenamiento. Escribiremos $(A^{(i)}, P^{(i)}, N^{(i)})$ para denotar el $i$-ésimo ejemplo de entrenamiento.

Nos queremos asegurar que una  imagen $A^{(i)}$ sea más cercana a la imagen Positiva $P^{(i)}$ que a la imagen Negativa $N^{(i)}$) por al menos un margen $\alpha$. Es decir:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

Ahora queremos minimizar esta función de coste triple:

$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Aquí usamos la notación "$[z]_+$" para denotar $max(z,0)$.  

Notas:
- El término (1) es la distancia al cuadrado entre la imagen ancla y la imagen positiva para un determinado trío, queremos que esta distancia sea pequeña.
- El término (2) es la distancia al cuadrado entre la imagen ancla y la imagen negativa para un determinado trío, queremos que esta distancia sea relativamente grande
- $\alpha$ se llama el margen y es un hiperparámetro que hay que elegir manualmente. En nuestro caso usamos $\alpha = 0.2$. 

La pérdida triple se implementa a continuación.

In [28]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_mean(tf.maximum(basic_loss,0.0))
    
    return loss

## 3 - Cargando un modelo pre-entrenado

FaceNet está entrenado para minimizar la función de pérdida triple. Pero debido a que este proceso de entrenamiento lleva mucho tiempo, nosotros no lo entrenaremos desde cero. En su lugar, cargamos un modelo previamente entrenado. Para ello debes ejecutar la siguiente celda, lo cual puede llevar algunos minutos.

In [29]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

A continuación se muestran algunos ejemplos de las distancias entre las codificaciones de tres personas.
<img src="images/distance_matrix.png" style="width:380px;height:200px;">
<br>
<caption><center> <u> <font color='purple'> **Figura 4**:</u> <br>  <font color='purple'> Ejemplo de las distancias entre las codificaciones de tres personas.</center></caption>

Veámos ahora cómo usar este modelo para llevar a cabo verificación facial y reconocimiento facial.

## 4 - Aplicando el modelo

Queremos construir un sistema que permita el acceso a nuestra casa usando un sistema de verificación facial. En concreto para admitir a una persona en nuestra casa deberá pasar una tarjeta de identificación para identificarse. El sistema se encargará de verificar si la persona que ha pasado la tarjeta es quien dice ser.

### 4.1 - Verificación facial

Empezamos construyendo una base de datos que contiene la codificación de cada persona que tiene permitido el acceso a nuestra casa. Para generar la codificación se utiliza la función `img_to_encoding(image_path, model)` que básicamente ejecuta el paso de propagación hacia delante del modelo en una imagen.

Ejecuta el siguiente código para construir la base de datos que se representa mediante un diccionario Python. Esta base de datos asigna un vector de tamaño 128 a cada cara. Las imágenes que se añaden a la base de datos están en la carpeta `images`, así que puedes añadir tus propias imágenes. 


In [30]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

La base de datos tiene como identificador el nombre (una mala idea, pero por simplicidad) lo hacemos así. 

Ahora cuando alguien se presenta en la entrada de nuestra casa y pasa su tarjeta de identificación (lo cual nos da su nombre) podemos buscar en la base de datos la codificación asociada a su nombre y comprobar si la persona que se encuentra en la puerta coincide con la que está en la tarjeta de identificación. 

La siguiente función comprueba si la imagen capturada por la puerta de nuestra casa (disponible en `image_path`) pertenece realmente a la persona que dice ser. Para ello:
1. Se calcula la codificación de la imagen capturada por la cámara de la casa.
2. Se calcula la distancia a la codificación de la imagen de la persona de la tarjeta en la base de datos.
3. Se abre la puerta si la distancia es menor que 0.7, en caso contrario la puerta no se abre.

In [33]:
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path,model)

    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist<0.7:
        print("Eres " + str(identity) + ", bienvenido")
        door_open = True
    else:
        print("No eres " + str(identity) + ", por favor márchate")
        door_open = False
        
    return dist, door_open

Younes está intentando entrar a la casa y la cámara toma una imagen de el("images/camera_0.jpg"). Vamos a ejectar nuestro algoritmo de verificación en dicha imagen.

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [34]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

Eres younes, bienvenido


(0.67291224, True)

Benoit no tiene permitido el acceso a nuestra casa, pero le ha robado la tarjeta a Kian y está intentando entrar. La cámara frontal de nuestra casa toma una imagen de Benoit ("images/camera_2.jpg). Veámos si Benoit puede entrar a nuestra casa.
<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [35]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

No eres kian, por favor márchate


(0.86543161, False)

### 4.2 - Reconocimiento facial

El sistema de verifiación facial está funcionando bastante bien. Pero debido a que a Kian le han robado su tarjeta, cuando intente volver, no podrá entrar. 

Para resolver este problema vamos a implantar un sistema de reconocimiento facial. De este modo no será necesario llevar una tarjeta de identificación nunca mas: toda persona autorizada podrá acercarse a nuestra casa, y la puerta de entrada se abrirá. 

Para ello vamos a implementar un sistema de reconocimiento facial que toma una imagen y comprueba si es de una de las personas autorizadas. Al contrario que en el caso anterior, no necesitamos ninguna información adicional. La siguiente función implementa dicha funcionalidad con los siguientes pasos:
1. Calcula la codificación objetivo de la imagen.
2. Encuentra la codificación en la base de datos que es más cercana a la codificación objetivo. Para ello:
    - Inicializa la variable `min_dist` a un número lo suficientemente grande. 
    - Se itera sobre el diccionario de la base de datos:
        - Se calcula la distancia L2 entre la codificación objetivo y la codificación actual. 
        - Si la distancia es menor que min_dist, se asigna dicha distancia a min_dist y se identifica por nombre.

In [38]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-database[name])

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.7:
        print("No esta en la base de datos.")
    else:
        print ("Eres " + str(identity) + ", la distancia es " + str(min_dist) + ", bienvenido")
        
    return min_dist, identity

Younes se encuentra en la puerta de entrada y la cámara toma una imagen de el. Veámos si el algoritmo es capaz de identificarlo.

In [39]:
who_is_it("images/camera_0.jpg", database, FRmodel)

Eres younes, la distancia es 0.672912, bienvenido


(0.67291224, 'younes')

Ya tenemos listos nuestros sistemas de verificación y reconocimiento facial. 

El algoritmo aquí presentado se puede mejorar de muchas maneras:
- Poniendo más imágenes de cada persona para incrementar la accuracy del modelo. 
- Recortar las imágenes para que contengan solo la cara con el objetivo de hacer el algoritmo más robusto.
- ...

### Referencias:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- The pretrained model we use is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 
